In [123]:
from kalmanfilter import kalmanfilter
from behaviour_based_navigation import compute_velocity
from sprite_routines import nao_robot

In [124]:
import numpy as np

In [125]:
def dynamic_model(state,input,A,B,noise):
    return A*state+B*input+noise

def measurement(state,C,noise):
    return C*state+noise

In [126]:
delta_t=0.05 # According to the sprite_routines defination

In [127]:
initState=np.matrix(np.array([[0.1,0.5]]).T)
initSigma=np.matrix(np.identity(initState.shape[0])*0.005)

In [128]:
A=np.matrix(np.identity(initState.shape[0]))
B=np.matrix(np.array([[np.sqrt(3)/2,np.sqrt(3)/2]]).T*delta_t)
C=np.matrix(np.identity(initState.shape[0]))
R=np.matrix(0.01*np.identity(initState.shape[0])) # epsilon noise (process noise)
Q=np.matrix(0.01*np.identity(initState.shape[0])) # delta noise (measurement noise)

In [129]:
states=[initState]
true_state=[initState]
mu_s=[initState]
sigma_s=[np.matrix([0,0,0,0]).reshape(2,2)]
mu=mu_s[0]
sigma=sigma_s[0]
samples=50
for t in range(samples):
    cur_state=states[t]
    true_cur_state=true_state[t]
    states.append(dynamic_model(cur_state,compute_velocity(cur_state[0,0],cur_state[1,0]),A,B,np.matrix(np.random.multivariate_normal([0,0],R).reshape(-1,1))))
    # mu,sigma=kalmanfilter(mu,sigma,compute_velocity(mu[0,0],mu[1,0]),cur_state,A,B,C,R,Q)
    mu,sigma=kalmanfilter(mu,sigma,compute_velocity(mu[0,0],mu[1,0]),measurement(true_cur_state,C, np.matrix(np.random.multivariate_normal([0,0],Q).reshape(-1,1)) ),A,B,C,R,Q)
    mu_s.append(mu)
    sigma_s.append(sigma)
    true_state.append(dynamic_model(true_cur_state,compute_velocity(true_cur_state[0,0],true_cur_state[1,0]),A,B,0))

In [130]:
from matplotlib import pyplot as plt

In [131]:
all_states=np.hstack(states)
all_true_states=np.hstack(true_state)
all_kalman_states=np.hstack(mu_s)

In [132]:
x=np.arange(samples+1)
all_sigmas=np.sqrt(np.array(sigma_s))

In [133]:
plt.plot(np.array(all_states[0,:]).squeeze(),label="Noisy")
plt.plot(np.array(all_true_states[0,:]).squeeze(),label="No noise")
plt.plot(np.array(all_kalman_states[0,:]).squeeze(),label="kalman")
plt.fill_between(x,np.array(all_kalman_states[0,:]).squeeze()-all_sigmas[:,0,0].squeeze(),np.array(all_kalman_states[0,:]).squeeze()+all_sigmas[:,0,0].squeeze(),alpha=0.3)

In [134]:
plt.legend()
plt.show()

In [78]:
all_sigmas.shape

(51, 2, 2)

In [85]:
plt.plot(x,np.array(all_states[1,:]).squeeze(),label="Noisy")
plt.plot(x,np.array(all_true_states[1,:]).squeeze(),label="No noise")
plt.plot(x,np.array(all_kalman_states[1,:]).squeeze(),label="kalman")
plt.fill_between(x,np.array(all_kalman_states[1,:]).squeeze()-all_sigmas[:,1,1].squeeze(),np.array(all_kalman_states[1,:]).squeeze()+all_sigmas[:,1,1].squeeze(),alpha=0.3)

In [86]:
plt.legend()
plt.show()